In [1]:
!pip install -qqq datasets==3.2.0 transformers trl peft accelerate bitsandbytes --progress-bar off
!pip install -qqq torch==2.6.0+cu124 flash-attn==v2.7.4.post1 --no-build-isolation --progress-bar off
!pip install -qqq sentence-transformers scikit-learn --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
import os

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# os.environ["FLASH_ATTENTION_FORCE_DISABLED"] = "1"

In [5]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from datasets import concatenate_datasets
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# prompt: connect gg drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
root_path = '/content/drive/MyDrive/RL/gpt2'

In [8]:
ds = load_dataset("yonatanko/Relationship_Advice")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

dev.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/210 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [9]:
ds = ds.rename_column("post", "prompt")

def get_answer(example):
  more_helpful_comment = example["more_helpful_comment"].lower().replace(" ","_")
  if example["more_helpful_comment"] == "comment_1":
    example["answer"] = example["comment_1"]
  else:
    example["answer"] = example["comment_2"]
  return example

ds = ds.map(get_answer)
ds = ds.remove_columns(["comment_1", "comment_2", "comment_1_label", "comment_2_label", "more_helpful_comment", "batch", "example_id"])

Map:   0%|          | 0/210 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [10]:
ds['all'] = concatenate_datasets([ds['train'], ds['test'], ds['validation']])

In [11]:
ds['all']

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 400
})

In [13]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set the padding token
# tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# if tokenizer.pad_token is not None and tokenizer.pad_token != tokenizer.eos_token:
#     model.resize_token_embeddings(len(tokenizer))

In [14]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 8,798,208 || all params: 502,830,976 || trainable%: 1.7497
None


In [53]:
# Reward function
# model_st = SentenceTransformer('all-MiniLM-L6-v2')
def reward_tfidf_similarity(prompts, completions, answer, **kwargs):
    # Ensure all elements are strings before passing to TfidfVectorizer
    corpus = [str(answer)] + [str(c) for c in completions]
    vectorizer = TfidfVectorizer().fit(corpus)
    vectors = vectorizer.transform(corpus)
    sims = cosine_similarity(vectors[1:], vectors[0])
    return [float(1 - s[0]) for s in sims]

# def reward_semantic_similarity(prompts, completions, answer, **kwargs):
#     # Encode answer once
#     answer_embedding = model_st.encode(answer, convert_to_tensor=True)
#     # Encode all completions at once
#     completion_embeddings = model_st.encode(completions, convert_to_tensor=True)

#     # Calculate cosine similarity between each completion and the answer
#     similarity_scores = util.cos_sim(completion_embeddings, answer_embedding).flatten()

#     # Convert to list of floats
#     return [float(1 - score) for score in similarity_scores.tolist()]

def reward_len(prompts, completions, answer, **kwargs):
    return [float(abs(len(completion) - len(answer))) for completion in completions]

In [16]:
os.environ["WANDB_DISABLED"] = "true"

In [22]:
training_args = GRPOConfig(
    output_dir=os.path.join(root_path,"GRPO"),
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=None,
    remove_unused_columns=False,
    logging_steps=1,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [54]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len,reward_tfidf_similarity],
    args=training_args,
    train_dataset=ds["all"],
    processing_class=tokenizer
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [55]:
# Train model
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 76.12 MiB is free. Process 6608 has 14.66 GiB memory in use. Of the allocated memory 13.89 GiB is allocated by PyTorch, and 661.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)